In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from newspaper import Article
import nltk
nltk.download('punkt')

# headers = {
#     'Accept-Encoding': 'gzip, deflate, br, zstd',
#     'Accept-Language': 'en-US,en;q=0.9',
#     'Upgrade-Insecure-Requests': '1',
#     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'Cache-Control': 'max-age=0'
# }

s = requests.Session()
s.max_redirects = 100

url_blogs = "https://blogs.nasa.gov/"
url_news = "https://www.nasa.gov/wp-json/nasa-hds/v1/content-lists?postType=post%2Cfeature%2Cpress-release%2Cimage-article&postId=383829&maxPages=2380&perPage=25&categories=&layout=list&showThumbnails=yes&showReadTime=yes&showExcerpts=yes&showContentTypeTags=yes&pageClicked="


# "https://www.nasa.gov/news/all-news/#"
# https://api.nasa.gov/
# https://www.nasa.gov/wp-json/wp/v2/pages/383829
# https://www.nasa.gov/wp-json/nasa-hds/v1/content-lists

[nltk_data] Downloading package punkt to /Users/katie/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# The retrieval of the links to the articles

# links = []

# # page number is limited by the request to 2381
# for num in range(1, 2381):
#     page = requests.get(url_news + str(num))
#     if page.status_code != 200:
#         f'Response failed with status {page.status_code}'
#     else:
#         soup = BeautifulSoup(json.loads(page.text)['html'], "html.parser")
#         articles = soup.find_all("a", class_="hds-content-item-heading")
        
#     for article in articles:
#         links.append(article.get('href'))

In [ ]:
# with open('nasa_article_links.txt', 'w') as f:
#     for link in links:
#         f.write("%s,\n" % link)

In [ ]:
# Getting the links collected by the previous line
get_links = []
with open('nasa_article_links.txt', 'r', newline="\n") as f:
    for line in f:
        get_links.append(line.strip(",\n"))

# Discarded for now
    # page = requests.get(link)  # , headers=headers
    # if page.status_code != 200:
    #     f'Response failed with status {page.status_code}'
    # else:
    #     soup = BeautifulSoup(page.text, "html.parser")
    #     text = soup.find_all("div" ,class_="entry-content")
    #     articles.append(text)

In [51]:
session = requests.session()

/relative-redirect/3
/relative-redirect/2
/relative-redirect/1
/get
None


In [57]:
links_to_use = get_links[:1000]

articles_df = pd.DataFrame(columns=['link', 'title', 'text', 'summary', 'authors', 'keywords',  'publish_date', 'top_image', 'images'])
articles_df['link'] = links_to_use
articles_df.set_index('link', inplace=True)

for link in links_to_use:
    skip = ['https://www.nasa.gov/image-article/aurora-over-idaho/']
    if link not in skip:
        r = session.get(link, allow_redirects=False)
        r.headers.get('location')
        for redirect in session.resolve_redirects(r, r.request):
            print(link)
            print(redirect.headers.get('location'))
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()

        articles_df.loc[link, 'title'] = article.title 
        articles_df.loc[link, 'text'] = article.text
        articles_df.loc[link, 'summary'] = article.summary
        articles_df.loc[link, 'authors'] = article.authors
        articles_df.loc[link, 'keywords'] = article.keywords
        articles_df.loc[link, 'publish_date'] = article.publish_date 
        articles_df.loc[link, 'top_image'] = article.top_image 
        articles_df.loc[link, 'images'] = article.images

In [58]:
articles_df

,title,text,summary,authors,keywords,publish_date,top_image,images
link,,,,,,,,
https://www.nasa.gov/news-release/nasa-leaders-to-host-agency-town-hall-on-artificial-intelligence/,NASA Leaders to Host Agency Town Hall on Artif...,Credit: NASA\n\nNASA Administrator Bill Nelson...,Credit: NASANASA Administrator Bill Nelson and...,[Tiernan P. Doyle],"[leaders, hall, intelligence, ai, chief, missi...",None,https://www.nasa.gov/wp-content/uploads/2024/0...,{https://science.nasa.gov/wp-content/uploads/2...
https://science.nasa.gov/science-research/science-enabling-technology/nasa-wildfire-digital-twin-pioneers-new-ai-models-and-streaming-data-techniques-for-forecasting-fire-and-smoke/,NASA “Wildfire Digital Twin” Pioneers New AI M...,“We want to be able to provide firefighters wi...,“We want to be able to provide firefighters wi...,[],"[wildfire, version, smoke, pioneers, twin, tim...",None,https://science.nasa.gov/wp-content/uploads/20...,{https://science.nasa.gov/wp-content/uploads/2...
https://science.nasa.gov/blogs/sols-4188-4190-aurora-watch-on-mars/,Sols 4188-4190: Aurora Watch on Mars,Contact science on “Tuolumne Meadows” and “Par...,"Our current plan skips sol 4188, as Earth pass...",[],"[watch, mastcam, atmospheric, bedrock, chemcam...",None,https://science.nasa.gov/wp-content/uploads/20...,{https://science.nasa.gov/wp-content/uploads/2...
https://science.nasa.gov/science-research/heliophysics/aurorasaurus-roars-during-historic-solar-storm/,Aurorasaurus Roars During Historic Solar Storm,At Aurorasaurus.org you’ll see the latest mode...,At Aurorasaurus.org you’ll see the latest mode...,[],"[alerts, storm, aurorasaurus, youll, test, vol...",None,https://science.nasa.gov/wp-content/uploads/20...,{https://science.nasa.gov/wp-content/uploads/2...
https://www.nasa.gov/news-release/nasa-selects-bae-systems-to-develop-ocean-color-instrument-for-noaa/,NASA Selects BAE Systems to Develop Ocean Colo...,"NASA, on behalf of the National Oceanic and At...",The GeoXO Ocean Color instrument (OCX) will mo...,[Abbey A. Donaldson],"[bae, operations, develop, satellite, support,...",None,https://www.nasa.gov/wp-content/uploads/2022/0...,{https://science.nasa.gov/wp-content/uploads/2...
...,...,...,...,...,...,...,...,...
https://www.nasa.gov/feature/special-report-x-59-rollout/,Special Report: X-59 Makes its Public Debut,NASA e-book\n\nQuieting the Boom\n\nKey resear...,NASA e-bookQuieting the BoomKey research that ...,[Jim Banke],"[debut, shaped, research, supersonic, x59, rep...",None,https://www.nasa.gov/wp-content/uploads/2024/0...,{https://www.nasa.gov/wp-content/uploads/2023/...
https://www.nasa.gov/news-release/expedition-69-crew-three-nasa-astronauts-to-share-mission-highlights/,"Expedition 69 Crew, Three NASA Astronauts to S...",Expedition 69 Flight Engineers play chess with...,The crew also will be available for interviews...,[Johnson Space Center Office Of Communications],"[center, crew, share, astronauts, aboard, 69, ...",None,https://www.nasa.gov/wp-content/uploads/2024/0...,{https://science.nasa.gov/wp-content/uploads/2...
https://www.nasa.gov/directorates/somd/launch-services-office/lsp/nasa-dod-seek-universities-to-develop-cubesats-for-launch-2/,"NASA, DOD Seek Universities to Develop CubeSat...",A set of NanoRacks CubeSats is photographed by...,A set of NanoRacks CubeSats is photographed by...,[Nasa Communications],"[develop, 2024, cubesats, satellite, force, la...",2024-01-16 16:13:39-05:00,https://www.nasa.gov/wp-content/uploads/2024/0...,{https://science.nasa.gov/wp-content/uploads/2...


In [59]:
articles_df.to_csv("articles.csv")